In [1]:
from wgan_gp import WGAN_GP
from load_dataset import parser, full_parser
import tensorflow as tf
from tensorflow import keras
import pickle

2022-09-29 15:37:54.292209: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-29 15:37:54.518208: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-29 15:37:54.518244: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-29 15:37:54.550798: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-29 15:37:55.475188: W tensorflow/stream_executor/platform/de

In [2]:
# Creating GAN
wgan_gp = WGAN_GP()

Instructions for updating:
Colocations handled automatically by placer.


2022-09-29 15:37:57.570757: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-29 15:37:57.570801: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-29 15:37:57.570822: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gaspard-HP-ZBook-15u-G2): /proc/driver/nvidia/version does not exist


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 16)        592       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 16)        0         
                                                                 
 dropout (Dropout)           (None, 32, 32, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        4640      
                                                                 
 zero_padding2d (ZeroPadding  (None, 17, 17, 32)       0         
 2D)                                                             
                                                                 
 batch_normalization (BatchN  (None, 17, 17, 32)       128       
 ormalization)                                          

ValueError: Cannot convert a partially known TensorShape (None, 64, 64, 4) to a Tensor.

In [ ]:
# Importing the dataset
tfile = "/home/gaspard/ETHZ/Master_Arbeit/acoupipe_datasets/training_1-100000_csmtriu_1src_he4.0625-1393.4375Hz_ds1-v001_13-Sep-2022.tfrecord"
dataset = tf.data.TFRecordDataset(filenames=[tfile])

# The parser convert the CSMTRIU into eigenvector (TODO: need to be extended to also extract the eigenvalues.)
dataset = dataset.map(full_parser)
#dataset = dataset.map(parser)

#  Creating the batches
dataset = dataset.shuffle(buffer_size=10).batch(wgan_gp.batch_size)


In [ ]:
# Training the GAN
wgan_gp.train(n_epochs=1245
, dataset=dataset)

In [ ]:
# backup wgan_gp
file_to_store = open("backup_wgan_gp.pickle", "wb")
pickle.dump(wgan_gp, file_to_store)
file_to_store.close()

In [ ]:
# get wgan from backup
file_to_read = open("backup_wgan_gp.pickle", "rb")
generated_backup = pickle.load(file_to_read)
file_to_read.close()

In [ ]:
import matplotlib.pyplot as plt

# Generating a sample:
generated = wgan_gp.get_sample()

if generated != None:
    evecs_r = generated[0,:,:,0]
    evecs_i = generated[0,:,:,1]
    evals_r = generated[0,:,:,2]
    evals_i = generated[0,:,:,3]

    fig = plt.figure(figsize=(15,15))
    plt.subplot(2,2,1)
    plt.imshow(evecs_r)
    plt.subplot(2,2,2)
    plt.imshow(evecs_i)
    plt.subplot(2,2,3)
    plt.imshow(evals_r)
    plt.subplot(2,2,4)
    plt.imshow(evals_i)


In [ ]:
# Backup of generated sample
"""
import pickle

file_to_store = open("backup_generated.pickle", "wb")
pickle.dump(generated, file_to_store)
file_to_store.close()
"""

In [ ]:
# restauring file from backup
file_to_read = open("backup_generated.pickle", "rb")
generated_backup = pickle.load(file_to_read)
file_to_read.close()

In [ ]:
from beamforming import beamform

beamform(generated_backup)

In [ ]:
from wgan_gp_copied_pasted import WGANGP

wgan = WGANGP()
wgan.train(epochs=30000, batch_size=32, sample_interval=100)


In [ ]:
from keras.layers import add, concatenate

print(dir(concatenate))